In [1]:
!pip install imbalanced-learn


In [2]:
import pandas as pd

url = "https://raw.githubusercontent.com/AnjulaMehto/Sampling_Assignment/main/Creditcard_data.csv"
data = pd.read_csv(url)

data.head()


,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,1
2,1,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


In [3]:
X = data.drop('Class', axis=1)
y = data['Class']

y.value_counts()


,count
Class,
0,763
1,9


In [7]:
from imblearn.over_sampling import RandomOverSampler, SMOTE
from imblearn.under_sampling import NearMiss
from imblearn.combine import SMOTEENN, SMOTETomek

samplers = {
    "Sampling1_RandomOver": RandomOverSampler(),
    "Sampling2_SMOTE": SMOTE(),
    "Sampling3_NearMiss": NearMiss(),
    "Sampling4_SMOTEENN": SMOTEENN(),
    "Sampling5_SMOTETomek": SMOTETomek()
}


In [8]:
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier

models = {
    "M1_Logistic": LogisticRegression(max_iter=2000),
    "M2_DecisionTree": DecisionTreeClassifier(),
    "M3_RandomForest": RandomForestClassifier(),
    "M4_SVM": SVC(),
    "M5_KNN": KNeighborsClassifier()
}


In [9]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import pandas as pd

results = pd.DataFrame(index=models.keys(), columns=samplers.keys())

for s_name, sampler in samplers.items():

    # apply sampling directly on original data
    X_s, y_s = sampler.fit_resample(X, y)

    X_train, X_test, y_train, y_test = train_test_split(
        X_s, y_s, test_size=0.3, random_state=42
    )

    for m_name, model in models.items():
        model.fit(X_train, y_train)
        pred = model.predict(X_test)
        acc = accuracy_score(y_test, pred) * 100
        results.loc[m_name, s_name] = round(acc,2)

results


,Sampling1_RandomOver,Sampling2_SMOTE,Sampling3_NearMiss,Sampling4_SMOTEENN,Sampling5_SMOTETomek
M1_Logistic,91.92,92.58,50.0,96.48,90.48
M2_DecisionTree,98.25,97.82,16.67,98.83,98.19
M3_RandomForest,99.78,99.13,16.67,99.71,99.77
M4_SVM,72.27,68.56,16.67,75.95,70.29
M5_KNN,98.47,86.46,83.33,94.13,82.99


In [10]:
best_sampling = results.idxmax(axis=1)
best_sampling


,0
M1_Logistic,Sampling4_SMOTEENN
M2_DecisionTree,Sampling4_SMOTEENN
M3_RandomForest,Sampling1_RandomOver
M4_SVM,Sampling4_SMOTEENN
M5_KNN,Sampling1_RandomOver
